In [ ]:
!pip install transformers peft datasets accelerate bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

**2. Drive 마운트**

In [ ]:
# 완전 초기화
import torch
import gc
if 'model' in globals():
    del model
if 'trainer' in globals():
    del trainer
torch.cuda.empty_cache()
gc.collect()

from google.colab import drive
import os
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/chattoner_finetuning"
os.makedirs(save_dir, exist_ok=True)
os.makedirs(f"{save_dir}/final_model4", exist_ok=True)
os.makedirs(f"{save_dir}/checkpoints", exist_ok=True)

print("초기화 및 Drive 마운트 완료")

Mounted at /content/drive
초기화 및 Drive 마운트 완료


**3. 모델 로딩**

In [ ]:
from huggingface_hub import login
login("YOUR_HUGGINGFACE_TOKEN")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

model_id = "google/gemma-2-2b-it"

# BitsAndBytesConfig 명시적 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    token=True
)
tokenizer.pad_token = tokenizer.eos_token

# 모델 로딩
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=True,
    torch_dtype=torch.float16,
    attn_implementation="eager"
)

# LoRA를 위한 모델 준비
model = prepare_model_for_kbit_training(model)

# LoRA 설정 안정화
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    modules_to_save=None,
)

# LoRA 적용
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("모델 설정 완료")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

trainable params: 1,597,440 || all params: 2,615,939,328 || trainable%: 0.0611
모델 설정 완료


**4. 데이터 로딩**

In [ ]:
import json
import re
from datasets import Dataset

# JSONL 파일 경로
jsonl_file_path = "/content/drive/MyDrive/chattoner_finetuning/merged_formal.jsonl"

def remove_image_content(text):
    text = re.sub(r'https?://[^\s]+\.(jpg|jpeg|png|gif|bmp|webp)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\[이미지[^\]]*\]', '', text)
    text = re.sub(r'\(이미지[^\)]*\)', '', text)
    text = re.sub(r'<img[^>]*>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def analyze_formal_document_style(text):
    """공적 문서의 말투 특성을 분석 (더 관대한 기준)"""

    formal_characteristics = {
        # 존댓말 표현
        'honorific_endings': len(re.findall(r'[가-힣]+(?:습니다|하겠습니다|드리겠습니다|해주십시오|하십시오|됩니다|있습니다)', text)),

        # 공식적 동사 표현
        'formal_verbs': len(re.findall(r'(?:시행|실시|추진|진행|검토|논의|협의|제안|건의|요청|승인|결정|확정|개최|참석|발표)', text)),

        # 문어체 표현
        'written_expressions': len(re.findall(r'(?:하며|되며|이며|그러나|또한|따라서|그러므로|한편|아울러|더불어|관련하여)', text)),

        # 공식적 명사
        'formal_nouns': len(re.findall(r'(?:사안|현안|방안|대안|정책|제도|규정|지침|기준|원칙|절차|과정|위원회|협의회|회의)', text)),

        # 정중한 표현
        'polite_expressions': len(re.findall(r'(?:부탁드리|말씀드리|안내드리|알려드리|양해|협조|참고|검토|감사드리)', text)),
    }

    # 비공식 요소 체크 (감점 요소)
    informal_penalty = 0
    if re.search(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002702-\U000027B0\U000024C2-\U0001F251]+', text):
        informal_penalty += 0.3  # 이모티콘
    if '!!' in text or '~~' in text:
        informal_penalty += 0.2  # 과도한 감정 표현
    if re.search(r'[가-힣]+[야|지|어](?=\s|$|[.!?])', text):
        informal_penalty += 0.3  # 반말
    if re.search(r'ㅋㅋ|ㅎㅎ|ㅠㅠ', text):
        informal_penalty += 0.2  # 인터넷 표현

    # 간단한 점수 계산 (0-1)
    base_score = (
        formal_characteristics['honorific_endings'] * 0.04 +
        formal_characteristics['formal_verbs'] * 0.03 +
        formal_characteristics['written_expressions'] * 0.05 +
        formal_characteristics['formal_nouns'] * 0.02 +
        formal_characteristics['polite_expressions'] * 0.04
    )

    final_score = max(0, base_score - informal_penalty)

    return {
        'formal_characteristics': formal_characteristics,
        'formal_score': final_score,
        'text_length': len(text.split()),
        'has_informal_elements': informal_penalty > 0
    }

def categorize_texts_by_quality(data):
    """텍스트를 품질별로 분류"""
    categorized = {
        'high_quality': [],      # 상위 20%
        'medium_quality': [],    # 중간 60%
        'low_quality': []        # 하위 20%
    }

    scored_texts = []

    for item in data:
        text = item['input']
        if len(text.split()) > 20:  # 최소 길이
            analysis = analyze_formal_document_style(text)
            scored_texts.append({
                'text': text,
                'analysis': analysis,
                'original_item': item
            })

    # 점수순으로 정렬
    scored_texts.sort(key=lambda x: x['analysis']['formal_score'], reverse=True)

    total = len(scored_texts)
    if total > 0:
        # 상위 20% (최소 20개, 최대 100개)
        high_end = max(20, min(100, int(total * 0.2)))
        categorized['high_quality'] = scored_texts[:high_end]

        # 중간 60% (최소 50개, 최대 200개)
        medium_start = high_end
        medium_end = min(total, medium_start + max(50, min(200, int(total * 0.6))))
        categorized['medium_quality'] = scored_texts[medium_start:medium_end]

        # 나머지는 low_quality
        categorized['low_quality'] = scored_texts[medium_end:]

    return categorized, scored_texts

def load_and_preprocess_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line.strip())

            if 'input' in item:
                item['input'] = remove_image_content(item['input'])
            if 'content' in item:
                item['content'] = remove_image_content(item['content'])

            processed_item = {
                "instruction": "다음은 연설문의 공적 문체 예시입니다.",
                "input": item.get('content', item.get('input', '')),
                "output": item.get('output', ''),
                "source": item.get('source', 'unknown')
            }

            if processed_item['input'].strip() and len(processed_item['input']) > 50:
                data.append(processed_item)

    return data

# 데이터 로딩
train_data = load_and_preprocess_data(jsonl_file_path)
print(f"전체 데이터 로딩 완료: {len(train_data)}개")

# 품질별로 분류
categorized_data, all_scored_texts = categorize_texts_by_quality(train_data)

print(f"\n=== 품질별 분류 결과 ===")
print(f"고품질 텍스트: {len(categorized_data['high_quality'])}개")
print(f"중품질 텍스트: {len(categorized_data['medium_quality'])}개")
print(f"저품질 텍스트: {len(categorized_data['low_quality'])}개")

# 점수 분포 확인
if all_scored_texts:
    scores = [item['analysis']['formal_score'] for item in all_scored_texts]
    print(f"\n=== Formal Score 분포 ===")
    print(f"최고 점수: {max(scores):.3f}")
    print(f"최저 점수: {min(scores):.3f}")
    print(f"평균 점수: {sum(scores)/len(scores):.3f}")

    # 상위 5개 예시
    print(f"\n=== 상위 5개 고품질 텍스트 ===")
    for i, item in enumerate(all_scored_texts[:5]):
        print(f"{i+1}. Score: {item['analysis']['formal_score']:.3f}")
        print(f"   텍스트: {item['text'][:100]}...")
        print(f"   특성: 존댓말({item['analysis']['formal_characteristics']['honorific_endings']}), "
              f"공식동사({item['analysis']['formal_characteristics']['formal_verbs']}), "
              f"공식명사({item['analysis']['formal_characteristics']['formal_nouns']})")
        print()

# 학습용 데이터셋 구성 (고품질 + 중품질 혼합)
training_items = []

# 고품질 텍스트는 3번 반복 (가중치 부여)
for item in categorized_data['high_quality']:
    for _ in range(3):
        training_items.append(item['original_item'])

# 중품질 텍스트는 1번
for item in categorized_data['medium_quality']:
    training_items.append(item['original_item'])

# 너무 적으면 저품질도 일부 포함
if len(training_items) < 150:
    # 저품질 중에서도 상위 일부 포함
    additional_items = categorized_data['low_quality'][:50]
    for item in additional_items:
        training_items.append(item['original_item'])

print(f"\n최종 학습 데이터셋: {len(training_items)}개 (고품질 데이터 가중치 적용)")

전체 데이터 로딩 완료: 402개

=== 품질별 분류 결과 ===
고품질 텍스트: 80개
중품질 텍스트: 200개
저품질 텍스트: 122개

=== Formal Score 분포 ===
최고 점수: 20.540
최저 점수: 0.000
평균 점수: 1.257

=== 상위 5개 고품질 텍스트 ===
1. Score: 20.540
   텍스트: 제4차 환경교육훈련 중기계획 (2022～2026) - 2021. 12. 환경부 국립환경인재개발원 순 서 제1장 환경교육훈련 중기계획 의의 1 제1절 배경 및 필요성 2 제2절 계획...
   특성: 존댓말(0), 공식동사(330), 공식명사(421)

2. Score: 17.210
   텍스트: 2023~2027년 제2차 심뇌혈관질환관리 종합계획[안] 2023. 7. 관계부처합동 목 차 Ⅰ. 수립 배경 1 “심뇌혈관질환”제대로 알기(Fact Sheet) Ⅱ. 그 간의 정책...
   특성: 존댓말(0), 공식동사(288), 공식명사(364)

3. Score: 15.440
   텍스트: 제1차 공공보건의료 기본계획 (2016∼2020) 2016. 3. 보 건 복 지 부 공공보건정책관 목 차 Ⅰ. 추진배경 1 Ⅱ. 공공보건의료정책 현황 및 여건진단 4 Ⅲ. 공공보건...
   특성: 존댓말(0), 공식동사(377), 공식명사(164)

4. Score: 13.400
   텍스트: 2016년도 발전소주변지역지원사업 사업계획 수립 및 관리지침(안) 2015. 10 산업통상자원부 목 차 제1장 총 칙 1 1. 목 적 1 2. 적용범위 1 3. 지원사업 신규신청 ...
   특성: 존댓말(4), 공식동사(293), 공식명사(158)

5. Score: 11.560
   텍스트: 참고자료 제4차 국가기술자격 제도발전 기본계획 (2018∼2022) 2018. 11. 관계부처 합동 순 서 Ⅰ. 추진배경 1 Ⅱ. 현황 및 평가 3 Ⅲ. 비전 및 전략 10 Ⅳ. ...
   특성: 존댓말(0), 공식동사(143), 공식명사(292)


최종

**5. 토크나이징**

In [ ]:
def create_smart_conversion_data(example):

    # 짧고 효과적인 Few-shot
    examples = [
        ("안녕해요! 😊", "안녕하십니까."),
        ("좋아요!! 👍", "좋습니다."),
        ("죄송해요 ㅠㅠ", "죄송합니다.")
    ]

    formal_text = example['input'][:200]

    # 명확한 변환 구조
    prompt = "말투 변환:\n"
    for inp, out in examples:
        prompt += f"{inp} → {out}\n"

    # 실제 변환 작업
    prompt += f"[비공식 텍스트] → {formal_text}"

    return prompt

def tokenize_function(example):
    text = create_smart_conversion_data(example)

    result = tokenizer(
        text,
        truncation=True,
        max_length=384,
        padding=False,
    )

    result["labels"] = result["input_ids"].copy()
    return result

# 데이터셋 재생성
if len(training_items) > 0:
    dataset = Dataset.from_list(training_items[:30])  # 더 줄임
    tokenized_dataset = dataset.map(
        tokenize_function,
        remove_columns=dataset.column_names,
    )
    print(f"단순 토크나이징 완료: {len(tokenized_dataset)}개")

    # 검증
    sample = tokenized_dataset[0]
    decoded = tokenizer.decode(sample['input_ids'])
    print(f"프롬프트:\n{decoded[:400]}...")

    # Loss 테스트
    model.eval()
    with torch.no_grad():
        test_inputs = {
            'input_ids': torch.tensor([sample['input_ids'][:100]]).to(model.device),
            'labels': torch.tensor([sample['labels'][:100]]).to(model.device)
        }
        test_outputs = model(**test_inputs)
        test_loss = test_outputs.loss.item()
        print(f"테스트 Loss: {test_loss}")
    model.train()

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

단순 토크나이징 완료: 30개
프롬프트:
<bos>말투 변환:
안녕해요! 😊 → 안녕하십니까.
좋아요!! 👍 → 좋습니다.
죄송해요 ㅠㅠ → 죄송합니다.
[비공식 텍스트] → 제4차 환경교육훈련 중기계획 (2022～2026) - 2021. 12. 환경부 국립환경인재개발원 순 서 제1장 환경교육훈련 중기계획 의의 1 제1절 배경 및 필요성 2 제2절 계획의 성격 및 범위 3 제2장 교육환경 분석 및 시사점 4 제1절 인재개발원 교육환경 분석 5 제2절 국내외 교육사례 분석 시사점 15 제3절 업무성과분석 밑 내외부 인터뷰 시사점 ...
테스트 Loss: 3.2612807750701904


**6. 학습 설정**

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import os

# wandb 비활성화
os.environ["WANDB_DISABLED"] = "true"

# 데이터 콜레이터
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

# 학습 설정
training_args = TrainingArguments(
    output_dir=f"{save_dir}/checkpoints",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    learning_rate=1e-4,
    weight_decay=0.01,
    fp16=True,
    logging_steps=1,
    save_strategy="epoch",
    save_total_limit=2,
    seed=42,
    dataloader_pin_memory=False,
    gradient_checkpointing=False,
    report_to="none",
    remove_unused_columns=False,
    max_grad_norm=1.0,
    dataloader_num_workers=0,
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("학습 설정 완료")

학습 설정 완료


/tmp/ipython-input-478106355.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


**7. 학습 시작**

In [ ]:
import time

print("10 에폭 학습 시작...")
start_time = time.time()

# 학습 실행
trainer.train()

end_time = time.time()
print(f"10 에폭 학습 완료! 소요 시간: {(end_time - start_time)/60:.2f}분")

# 학습 loss 출력
if trainer.state.log_history:
    final_loss = trainer.state.log_history[-1].get('train_loss', 'N/A')
    print(f"최종 loss: {final_loss}")

10 에폭 학습 시작...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.725300
2,2.746700
3,2.864000
4,2.770000
5,2.737300
6,2.850300
7,2.765200
8,2.739500
9,2.835200
10,2.686800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

10 에폭 학습 완료! 소요 시간: 1.19분
최종 loss: 2.6514980673789976


In [ ]:
# 현재 학습 상태 확인
if hasattr(trainer.state, 'log_history'):
    print(f"현재까지 로그 수: {len(trainer.state.log_history)}")
    if trainer.state.log_history:
        latest_log = trainer.state.log_history[-1]
        print(f"최근 로그: {latest_log}")

    print(f"현재 에폭: {trainer.state.epoch}")
    print(f"현재 스텝: {trainer.state.global_step}")
    print(f"최대 스텝: {trainer.state.max_steps}")

# GPU 메모리 사용량 확인
import torch
if torch.cuda.is_available():
    print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"GPU 메모리 예약량: {torch.cuda.memory_reserved()/1024**3:.2f} GB")

현재까지 로그 수: 41
최근 로그: {'train_runtime': 70.999, 'train_samples_per_second': 4.225, 'train_steps_per_second': 0.563, 'total_flos': 807520420666368.0, 'train_loss': 2.6514980673789976, 'epoch': 10.0, 'step': 40}
현재 에폭: 10.0
현재 스텝: 40
최대 스텝: 40
GPU 메모리 사용량: 3.35 GB
GPU 메모리 예약량: 9.30 GB




---



**8. 모델 저장**

In [ ]:
import os
import json
from datetime import datetime

# 1. 모델과 토크나이저 저장
print("=== 모델 저장 시작 ===")

# LoRA 어댑터와 토크나이저 저장
trainer.save_model(f"{save_dir}/final_model4")
tokenizer.save_pretrained(f"{save_dir}/final_model4")

# 베이스 모델 정보도 함께 저장
model_info = {
    "base_model_id": "google/gemma-2-2b-it",
    "model_type": "gemma2",
    "lora_config": {
        "r": 8,
        "lora_alpha": 16,
        "target_modules": ["q_proj", "v_proj"],
        "lora_dropout": 0.1,
        "bias": "none",
        "task_type": "CAUSAL_LM"
    },
    "training_info": {
        "num_epochs": 10,
        "learning_rate": 1e-4,
        "batch_size": 1,
        "gradient_accumulation_steps": 16,
        "max_length": 384,
        "training_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "dataset_size": len(tokenized_dataset)
    }
}

# 모델 정보를 JSON 파일로 저장
with open(f"{save_dir}/final_model/model_info.json", 'w', encoding='utf-8') as f:
    json.dump(model_info, f, indent=2, ensure_ascii=False)

print(f"모델 저장 완료: {save_dir}/final_model")

# 2. 저장된 파일 상세 확인
print("\n=== 저장된 파일 확인 ===")
saved_files = os.listdir(f"{save_dir}/final_model")
total_size = 0

for file in saved_files:
    file_path = f"{save_dir}/final_model/{file}"
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path)
        total_size += size
        print(f"📄 {file}: {size/1024/1024:.2f} MB")

print(f"\n총 저장 용량: {total_size/1024/1024:.2f} MB")

# 3. 필수 파일 존재 확인
required_files = [
    "adapter_config.json",
    "adapter_model.safetensors",
    "tokenizer.json",
    "tokenizer_config.json",
    "model_info.json"
]

missing_files = []
for file in required_files:
    if file not in saved_files:
        missing_files.append(file)

if missing_files:
    print(f"⚠️  누락된 파일: {missing_files}")
else:
    print("✅ 모든 필수 파일이 정상적으로 저장되었습니다!")

=== 모델 저장 시작 ===
모델 저장 완료: /content/drive/MyDrive/chattoner_finetuning/final_model

=== 저장된 파일 확인 ===
📄 training_args.bin: 0.01 MB
📄 tokenizer.model: 4.04 MB
📄 tokenizer.json: 32.77 MB
📄 model_info.json: 0.00 MB
📄 README.md: 0.00 MB
📄 adapter_config.json: 0.00 MB
📄 chat_template.jinja: 0.00 MB
📄 tokenizer_config.json: 0.04 MB
📄 special_tokens_map.json: 0.00 MB
📄 adapter_model.safetensors: 12.26 MB

총 저장 용량: 49.13 MB
✅ 모든 필수 파일이 정상적으로 저장되었습니다!


In [ ]:
# 모델 저장 (어댑터만)
trainer.save_model(f"{save_dir}/final_model4")
tokenizer.save_pretrained(f"{save_dir}/final_model4")

print(f"모델 저장 완료: {save_dir}/final_model4")

# 저장된 파일 확인
import os
saved_files = os.listdir(f"{save_dir}/final_model4")
print(f"저장된 파일들: {saved_files}")

모델 저장 완료: /content/drive/MyDrive/chattoner_finetuning/final_model4
저장된 파일들: ['README.md', 'adapter_model.safetensors', 'adapter_config.json', 'chat_template.jinja', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.model', 'tokenizer.json', 'training_args.bin']
